In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
Alpha 提交脚本

本脚本基于 machine_lib.py, config.py, 和 succeselfcon.ipynb 作为蓝本编写。

功能：
1. 读取 'unsubmitted_alphas_list.csv' 文件。
2. 根据 'sharpe' 列从低到高排序。
3. 依次提交 '未提交alpha_id' 列中的 Alpha。
4. 成功提交 7 个 Alpha 后自动停止。
5. 使用 'machine_lib.py' 中的 login() 函数进行认证。
"""

import os
import pandas as pd
import requests
import time
import logging

# --- 蓝本模块导入 ---
# 尝试从蓝本文件中导入必要的函数和变量
try:
    from machine_lib import login
    
except ImportError as e:
    print(f"错误：无法导入蓝本模块: {e}")
    print("请确保此脚本 (submit_alphas.py) 与 machine_lib.py 和 config.py 位于同一目录中。")
    exit(1)
# --- 蓝本模块导入结束 ---


# --- 全局配置 ---

# 根据 succeselfcon.ipynb 构建CSV文件路径
RECORDS_PATH = r"D:\python\selfcor\records";
# *** 注意：已确认使用您修改后的文件 ***
# 源文件路径
CSV_FILE_PATH = os.path.join(RECORDS_PATH, "check_results.csv")

# 提交的 Alpha ID 所在的列名
ALPHA_ID_COLUMN = "未提交alpha_id"

# 排序依据的列名
SORT_BY_COLUMN = "sharpe"

# 提交API端点 (根据您的指定)
SUBMIT_URL_TEMPLATE = "https://api.worldquantbrain.com/alphas/{alpha_id}/submit"

# 每次提交之间的基础延迟（秒），避免过于频繁的请求
BASE_SUBMIT_DELAY_SEC = 2

# 配置日志
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)


def submit_alpha(session, alpha_id):
    """
    使用认证会话提交单个 Alpha。
    
    *** 已修改：新的重试逻辑 ***
    1. 持续重试 20 秒。
    2. 如果 20 秒内所有尝试（包括API速率限制等待）都未成功，则跳过。

    :param session: 已认证的 requests.Session 对象
    :param alpha_id: 要提交的 Alpha ID
    :return: True (如果成功) / False (如果最终失败)
    """
    submit_url = SUBMIT_URL_TEMPLATE.format(alpha_id=alpha_id)
    logger.info(f"  -> 准备 POST 到: {submit_url}")

    # --- 新的重试配置 (根据要求修改) ---
    TOTAL_TIMEOUT_SEC = 20      # *** 修改点：总超时改为 20 秒 ***
    RETRY_POLL_DELAY_SEC = 5    # 瞬时失败后，重试前的等待时间
    # --- 结束修改 ---

    # --- 内部辅助函数，执行单次尝试 ---
    def attempt_submit():
        """
        执行一次提交尝试。
        返回:
        - "SUCCESS": 2xx 成功
        - "PERMANENT_FAIL": 404, 409 (永久失败，不应重试)
        - "RATE_LIMIT": 收到 Retry-After 头部 (返回等待秒数)
        - "TRANSIENT_FAIL": 403, 5xx, 网络错误 (可重试)
        """
        try:
            headers = {
                'Origin': 'https://platform.worldquantbrain.com',
                'Referer': 'https://platform.worldquantbrain.com/', # 匹配您浏览器中的 Referer
                'accept': 'application/json;version=2.0', # 必须添加这个
                'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36 Edg/141.0.0.0' # 最好也加上
            }
            # *** 修改点：单次请求超时改为 15 秒 (必须小于 TOTAL_TIMEOUT_SEC) ***
            response = session.post(submit_url, headers=headers, timeout=15)

            # 1. 检查 API 速率限制 (Retry-After)
            if "retry-after" in response.headers:
                try:
                    retry_seconds = float(response.headers["Retry-After"]) + 1 # 增加1秒缓冲
                except ValueError:
                    logger.warning("  -> 收到无效的 'Retry-After' 头部，使用默认延迟。")
                    retry_seconds = RETRY_POLL_DELAY_SEC
                logger.warning(f"  -> 收到 API 速率限制。等待 {retry_seconds:.1f} 秒后重试...")
                return "RATE_LIMIT", retry_seconds

            # 2. 检查成功
            if response.status_code in [200, 201, 202]:
                logger.info(f"  -> 成功提交！状态码: {response.status_code}")
                return "SUCCESS", None

            # 3. 检查已知的永久错误
            elif response.status_code == 409: # 409 Conflict
                logger.warning(f"  -> 提交失败 (状态码 409 Conflict)。永久失败：Alpha 已被提交或不符合资格。")
                return "PERMANENT_FAIL", None
            elif response.status_code == 404: # 404 Not Found
                logger.error(f"  -> 提交失败 (状态码 404 Not Found)。永久失败：Alpha ID '{alpha_id}' 不存在。")
                return "PERMANENT_FAIL", None
            
            # 4. 检查其他瞬时错误 (包括 403 Forbidden)
            elif response.status_code == 403:
                logger.error(f"  -> 提交失败 (状态码 403 Forbidden)。权限不足或认证失败。(将重试)")
                return "TRANSIENT_FAIL", None
            else:
                logger.error(f"  -> 提交失败。状态码: {response.status_code}。响应: {response.text[:200]}... (将重试)")
                return "TRANSIENT_FAIL", None

        except requests.exceptions.Timeout:
            logger.error(f"  -> 提交时发生请求超时 (超过 {15} 秒)。(将重试)")
            return "TRANSIENT_FAIL", None
        except requests.exceptions.RequestException as e:
            logger.error(f"  -> 提交时发生网络或请求错误: {e}。(将重试)")
            return "TRANSIENT_FAIL", None
        except Exception as e:
            # 捕获其他意外错误
            logger.error(f"  -> 提交时发生意外错误: {e}。(将重试)")
            return "TRANSIENT_FAIL", None

    # --- 新的重试逻辑 (20 秒超时) ---
    logger.info(f"  -> Alpha {alpha_id}: 开始 {TOTAL_TIMEOUT_SEC} 秒提交窗口...")
    start_time = time.time()
    
    while (time.time() - start_time) < TOTAL_TIMEOUT_SEC:
        status, data = attempt_submit()
        
        if status == "SUCCESS": 
            return True
        if status == "PERMANENT_FAIL": 
            return False

        # 确定等待时间
        wait_time = 0
        if status == "RATE_LIMIT":
            try:
                wait_time = float(data)
            except (ValueError, TypeError):
                logger.warning(f"  -> 收到无效的 'Retry-After' 值: {data}。使用默认值 {RETRY_POLL_DELAY_SEC} 秒。")
                wait_time = RETRY_POLL_DELAY_SEC
        elif status == "TRANSIENT_FAIL":
            wait_time = RETRY_POLL_DELAY_SEC
        
        # 检查等待是否会超出总超时时间
        # (time.time() + wait_time) 是预期的结束时间
        # (start_time + TOTAL_TIMEOUT_SEC) 是绝对的截止时间
        if (time.time() + wait_time) > (start_time + TOTAL_TIMEOUT_SEC):
            logger.warning(f"  -> Alpha {alpha_id}: 下次重试 (等待 {wait_time:.1f} 秒) 将超出 {TOTAL_TIMEOUT_SEC} 秒的总时限。")
            break # 超时，退出重试循环

        if wait_time > 0:
            # 确保睡眠时间不会超过剩余时间
            remaining_time = (start_time + TOTAL_TIMEOUT_SEC) - time.time()
            if wait_time > remaining_time:
                break # 等待时间超过总时限，直接跳出
            time.sleep(wait_time)

    # --- 最终失败 ---
    logger.error(f"  -> Alpha {alpha_id} 在 {TOTAL_TIMEOUT_SEC} 秒重试后最终失败。跳过此 Alpha。")
    return False

def main():
    """
    主执行函数
    """
    logger.info("--- Alpha 提交脚本启动 ---")
    
    # *** 新增：定义提交数量限制 ***
    SUBMISSION_LIMIT = 9
    
    # 步骤 1: 登录
    logger.info("正在登录 WorldQuant BRAIN (调用 machine_lib.py)...")
    try:
        session = login() #
        if not isinstance(session, requests.Session):
            logger.critical("登录失败，未返回有效的 Session 对象。脚本终止。")
            return
    except Exception as e:
        logger.critical(f"登录时发生致命错误: {e}。脚本终止。")
        return
    
    logger.info("登录成功。")

    # 步骤 2: 读取结果文件
    logger.info(f"正在读取结果文件: {CSV_FILE_PATH}")
    if not os.path.exists(CSV_FILE_PATH):
        logger.critical(f"错误：找不到文件 {CSV_FILE_PATH}。")
        # *** 修改：更新提示信息 ***
        logger.critical("请先运行[那个生成列表的脚本] 生成 unsubmitted_alphas_list.csv 文件。脚本终止。")
        return
        
    try:
        df = pd.read_csv(CSV_FILE_PATH,encoding='utf-8')
        
        # 验证必要的列是否存在
        if ALPHA_ID_COLUMN not in df.columns or SORT_BY_COLUMN not in df.columns:
            logger.critical(f"文件 {CSV_FILE_PATH} 缺少 '{ALPHA_ID_COLUMN}' 或 '{SORT_BY_COLUMN}' 列。脚本终止。")
            return
            
    except Exception as e:
        logger.critical(f"读取或解析 CSV 文件时出错: {e}。脚本终止。")
        return

    # 步骤 3: 按 Sharpe 排序 (从低到高)
    logger.info(f"正在根据 '{SORT_BY_COLUMN}' 列（升序）进行排序...")
    df_sorted = df.sort_values(by=SORT_BY_COLUMN, ascending=True)
    
    alpha_list_to_submit = df_sorted[[ALPHA_ID_COLUMN, SORT_BY_COLUMN]].dropna()
    total_count = len(alpha_list_to_submit)
    
    logger.info(f"文件加载并排序完毕。共找到 {total_count} 个有效的 Alpha 准备提交。")
    if total_count == 0:
        logger.warning("没有找到可提交的 Alpha。脚本执行完毕。")
        return

    # *** 新增：成功提交计数器 ***
    success_count = 0
    
    # 步骤 4: 遍历并提交
    for i, row in enumerate(alpha_list_to_submit.itertuples(), 1):
        alpha_id = row._asdict()[ALPHA_ID_COLUMN]
        sharpe_value = row._asdict()[SORT_BY_COLUMN]
        
        # *** 修改：更新日志以显示计数器 ***
        logger.info(f"\n--- (已成功 {success_count}/{SUBMISSION_LIMIT}) --- 正在处理第 {i}/{total_count} 个 Alpha ---")
        logger.info(f"Alpha ID: {alpha_id} (Sharpe: {sharpe_value:.4f})")
        
        try:
            success = submit_alpha(session, alpha_id)
            
            if success:
                logger.info(f"成功完成 Alpha {alpha_id} 的提交。")
                
                # *** 新增：更新计数器并检查限制 ***
                success_count += 1
                logger.info(f"--- 成功提交总数: {success_count}/{SUBMISSION_LIMIT} ---")
                
                if success_count >= SUBMISSION_LIMIT:
                    logger.info(f"已成功提交 {SUBMISSION_LIMIT} 个 Alpha，达到数量限制。停止脚本。")
                    break # 退出 for 循环
                # *** 修改结束 ***
                    
            else:
                logger.warning(f"Alpha {alpha_id} 提交失败或被跳过。")
                
            # 在两次提交之间添加礼貌性延迟
            logger.info(f"等待 {BASE_SUBMIT_DELAY_SEC} 秒后继续...")
            time.sleep(BASE_SUBMIT_DELAY_SEC)
            
        except KeyboardInterrupt:
            logger.warning("\n检测到用户中断 (Ctrl+C)。正在停止提交循环...")
            break
        except Exception as e:
            logger.error(f"处理 Alpha {alpha_id} 时发生严重错误: {e}")
            logger.info("等待 10 秒后尝试继续提交下一个 Alpha...")
            time.sleep(10)

    # *** 修改：更新最终的日志 ***
    logger.info(f"--- Alpha 提交脚本执行完毕 (共成功提交 {success_count} 个) ---")

if __name__ == "__main__":
    main()

2025-10-28 21:39:51,217 - INFO - --- Alpha 提交脚本启动 ---
2025-10-28 21:39:51,219 - INFO - 正在登录 WorldQuant BRAIN (调用 machine_lib.py)...


b'{"user":{"id":"ZY20347"},"token":{"expiry":14400.0},"permissions":["TUTORIAL","WORKDAY"]}'


2025-10-28 21:39:52,796 - INFO - 登录成功。
2025-10-28 21:39:52,797 - INFO - 正在读取结果文件: D:\python\selfcor\records\check_results.csv
2025-10-28 21:39:52,809 - INFO - 正在根据 'sharpe' 列（升序）进行排序...
2025-10-28 21:39:52,817 - INFO - 文件加载并排序完毕。共找到 68 个有效的 Alpha 准备提交。
2025-10-28 21:39:52,820 - INFO - 
--- (已成功 0/9) --- 正在处理第 1/68 个 Alpha ---
2025-10-28 21:39:52,820 - INFO - Alpha ID: ak6ZA9G5 (Sharpe: 1.4500)
2025-10-28 21:39:52,821 - INFO -   -> 准备 POST 到: https://api.worldquantbrain.com/alphas/ak6ZA9G5/submit
2025-10-28 21:39:52,821 - INFO -   -> Alpha ak6ZA9G5: 开始 20 秒提交窗口...
2025-10-28 21:39:53,384 - WARNING -   -> 收到 API 速率限制。等待 2.0 秒后重试...
2025-10-28 21:39:56,596 - ERROR -   -> 提交失败。状态码: 400。响应: <html>
<head><title>400 The plain HTTP request was sent to HTTPS port</title></head>
<body>
<center><h1>400 Bad Request</h1></center>
<center>The plain HTTP request was sent to HTTPS port</center>... (将重试)
2025-10-28 21:40:01,909 - ERROR -   -> 提交失败 (状态码 403 Forbidden)。权限不足或认证失败。(将重试)
2025-10-28 21:40:07